In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pandas as pd

In [6]:
# Upload charity_data.csv
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving charity_data.csv to charity_data (1).csv
User uploaded file "charity_data.csv" with length 3605168 bytes


In [7]:
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [8]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9", "T13", "T12", "T2", "T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Filter to group Classifications with less than 1883 together by using the same logic as above and reset index.
other_class = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<1883].reset_index()

# Only use the 'index' or Classification in the new variable name.
classifications_to_replace = other_class["index"]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns = "IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 84, activation = "sigmoid", input_dim = 43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 50, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 84)                3696      
                                                                 
 dense_4 (Dense)             (None, 50)                4250      
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7,997
Trainable params: 7,997
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5813 - accuracy: 0.7122
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5632 - accuracy: 0.7266
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5598 - accuracy: 0.7266
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5570 - accuracy: 0.7286
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5560 - accuracy: 0.7281
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7304
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7285
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5534 - accuracy: 0.7307
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5522 - accuracy: 0.7295
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5514 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5529 - accuracy: 0.7269 - 462ms/epoch - 2ms/step
Loss: 0.5529296398162842, Accuracy: 0.7268804907798767


In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units = 84, activation = "tanh", input_dim = 43))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units = 80, activation = "tanh"))

# Output layer
nn2.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 84)                3696      
                                                                 
 dense_7 (Dense)             (None, 80)                6800      
                                                                 
 dense_8 (Dense)             (None, 1)                 81        
                                                                 
Total params: 10,577
Trainable params: 10,577
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5751 - accuracy: 0.7193
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5608 - accuracy: 0.7277
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7291
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5544 - accuracy: 0.7290
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7296
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7296
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7313
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7320
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7325
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accura

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5538 - accuracy: 0.7297 - 436ms/epoch - 2ms/step
Loss: 0.5538078546524048, Accuracy: 0.72967928647995


In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units = 84, activation = "relu", input_dim = 43))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units = 80, activation = "relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units = 50, activation = "sigmoid"))

# Output layer
nn3.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 84)                3696      
                                                                 
 dense_10 (Dense)            (None, 80)                6800      
                                                                 
 dense_11 (Dense)            (None, 50)                4050      
                                                                 
 dense_12 (Dense)            (None, 1)                 51        
                                                                 
Total params: 14,597
Trainable params: 14,597
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5724 - accuracy: 0.7186
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5570 - accuracy: 0.7279
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7301
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7320
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7323
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7306
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7324
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accura

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5627 - accuracy: 0.7318 - 867ms/epoch - 3ms/step
Loss: 0.562749445438385, Accuracy: 0.7317784428596497


In [26]:
# Export our model to HDF5 file
nn3.save('AlphabetSoupCharity_Optimization.h5')